<a href="https://colab.research.google.com/github/VVdovichev/NLP_GB/blob/main/NLP_hw_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Урок 2. Создание признакового пространства

In [70]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-10-24 04:51:05--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-10-24 04:51:05--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1cd4912bfe0e2fa1cd89e2ba05.dl.dropboxusercontent.com/cd/0/inline/BvYaFQanm9kW5Bv6SUTyUkoM6O1TOQFetQ55atCdK13rlax_xlhsmZhjP7WMIJrn28KPOFIfL3NXxh-_LorSfZFuzRXMJzSo-7ZEA3ESM6W5DT-QfwcYJBKzX-mPluBbWJHjxEsZ1fcV011KI7cy5ERVtUWzo3bIdVhG1Xsd_HVAiw/file# [following]
--2022-10-24 04:51:06--  https://uc1cd4912bfe0e2fa1cd89e2ba05.dl.dropboxusercontent.com/cd/0/inline/BvYaFQanm9kW5Bv6SUTyUkoM6O1TOQFetQ55atCdK13rlax_xlhsmZhjP7WMIJrn28KPOFIfL3NXxh-_

In [71]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus.reader.tagged import word_tokenize
import numpy as np
import pandas as pd

from string import punctuation
from html import unescape

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

import torch
from tqdm import tqdm


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [72]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
data = pd.concat([positive, negative]).reset_index(drop=True)
data.tail(10)

,text,label
226824,"@ivanenko14 и у меня также, только будильник е...",negative
226825,Когда же я вернусь к тебе(( http://t.co/36qnCO...,negative
226826,"RT @qelasocadij: Скажите пожалуйста, как у чел...",negative
226827,У нас физ ра на улице\nПака линт:(\nЧерез 45 м...,negative
226828,"Нас сегодня отказались принять в сад, типа мы ...",negative
226829,Но не каждый хочет что то исправлять:( http://...,negative
226830,скучаю так :-( только @taaannyaaa вправляет мо...,negative
226831,"Вот и в школу, в говно это идти уже надо(",negative
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
226833,Такси везет меня на работу. Раздумываю приплат...,negative


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


Задание 1.
Задание: обучите три классификатора:

1) на токенах с высокой частотой

2) на токенах со средней частотой

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

In [74]:
x_train, x_test, y_train, y_test = train_test_split(data.text, data.label)

In [75]:
noise = stopwords.words('russian') + list(punctuation)
cnvec_high = CountVectorizer(min_df=1, max_df=0.9, tokenizer=word_tokenize, stop_words=noise)
cnvec_mid = CountVectorizer(min_df=1e-3, max_df=0.7, tokenizer=word_tokenize, stop_words=noise)
cnvec_low = CountVectorizer(min_df=1e-5, max_df=0.3, tokenizer=word_tokenize, stop_words=noise)

In [76]:
bow_cnvec_high = cnvec_high.fit_transform(x_train)
lr = LogisticRegression(random_state=42)
lr.fit(bow_cnvec_high, y_train)
pred_high = lr.predict(cnvec_high.transform(x_test))
print(classification_report(pred_high, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

    negative       0.80      0.77      0.78     29433
    positive       0.76      0.80      0.78     27276

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



In [77]:
bow_cnvec_mid = cnvec_mid.fit_transform(x_train)
lr.fit(bow_cnvec_mid, y_train)
pred_mid = lr.predict(cnvec_mid.transform(x_test))
print(classification_report(pred_mid, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

    negative       0.74      0.69      0.72     30137
    positive       0.68      0.73      0.70     26572

    accuracy                           0.71     56709
   macro avg       0.71      0.71      0.71     56709
weighted avg       0.71      0.71      0.71     56709



In [78]:
bow_cnvec_low = cnvec_low.fit_transform(x_train)
lr.fit(bow_cnvec_low, y_train)
pred_low = lr.predict(cnvec_low.transform(x_test))
print(classification_report(pred_low, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

    negative       0.79      0.76      0.78     29204
    positive       0.76      0.79      0.77     27505

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



Наиболее выжными для классификации оказались токены с высокой и низкой частотой.

###Задание 2.
найти фичи с наибольшей значимостью, и вывести их

In [79]:
cnvec_high.get_feature_names()[:10]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


["''", '++', '+++', '+++++', '++++++++', '+++лучше', '+-', '+.по', '+/-', '+0']

###Задание 3.
1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера

3) убедиться что для сетки нет переобучения

In [80]:
# 2
vct = HashingVectorizer(analyzer='char', n_features=2*100)
bow_vct = vct.fit_transform(x_train)
lr.fit(bow_vct, y_train)
pred = lr.predict(vct.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

    negative       0.99      1.00      0.99     27768
    positive       1.00      0.99      0.99     28941

    accuracy                           0.99     56709
   macro avg       0.99      0.99      0.99     56709
weighted avg       0.99      0.99      0.99     56709

